## Initial setup

In [ ]:
import os
import wandb
os.environ['WANDB_DISABLE_CODE'] = 'True'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['WANDB_NOTEBOOK_NAME']= 'EXP - dataloader'
os.environ['WANDB_DISABLE_CODE'] = 'True'
wandb.login()

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tqdm.notebook import tqdm
from yacs.config import CfgNode as CN
from imutils import paths

import matplotlib.pyplot as plt
import numpy as np
import losses
import random
import time
import cv2
import tensorflow as tf
print(tf.__version__)

tf.random.set_seed(666)
np.random.seed(666)

#### Datasets


In [ ]:
args = CN()
args.DATASET                  = CN()

args.EPOCHS                   = 100
args.BACKBONE                 = 'Resnet50'
args.LR                       = 0.001
args.BATCH_SIZE               = 210
args.DECAY_STEPS              = 1000
args.LOG_EVERY                = 20 # STEPS
args.CHECKPOINT_EVERY         = 50 # EPOCHS
args.IMAGE_SHAPE              = 160
args.DATASET.NAME             = 'Facesemore'
args.DATASET.TOTAL_CLASSES    = 85742
args.NUMBER_OF_PEOPLE         = 45
args.NUMBER_IMAGES_PER_PEOPLE = 40
args.MODEL_NAME               = args.BACKBONE + '_' + args.DATASET.NAME+'_' + str(args.LR)
args.DATASET.BASE_PATH        = '/mnt/shared_folder/faces_emore/imgs/'
args.LOG_DIR                  = '/mnt/rahulkanojia/Logs/Face_Hyper/Contrastive/' + args.MODEL_NAME
args.SAVE_DIR                 = '/mnt/rahulkanojia/Models/Face_Hyper/Contrastive/' + args.MODEL_NAME

import os
if not os.path.exists(args.LOG_DIR):
    os.makedirs(args.LOG_DIR)
if not os.path.exists(args.SAVE_DIR):
    os.makedirs(args.SAVE_DIR)

In [ ]:
print(args)

In [ ]:
import logging
logging.basicConfig(filename=args.LOG_DIR+'/'+ args.MODEL_NAME + '.log',level=logging.DEBUG)
logging.info('Arguments of Experiment'+ '\n'+ str(args) + '\n')

In [ ]:
# def load_data(t):
#     label = str(t.numpy()).split('/')[-2]
#     image_pixels = tf.image.decode_jpeg(tf.io.read_file(t))
#     image_pixels = tf.image.resize(image_pixels, [160,160])    
#     return image_pixels, label #return t.numpy().decode('utf-8').upper()
# AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
# for i in range(0,args.DATASET.TOTAL_CLASSES, args.NUMBER_OF_PEOPLE):
    
#     curr_class = i, i + args.NUMBER_OF_PEOPLE
    
#     images_path = []
#     for folder in range(i, i + args.NUMBER_OF_PEOPLE):
        
#         images_in_folder = len(os.listdir(args.DATASET.BASE_PATH+str(folder)))
#         images_in_folder = images_in_folder if images_in_folder < args.NUMBER_IMAGES_PER_PEOPLE else args.NUMBER_IMAGES_PER_PEOPLE
        
#         for path in os.listdir(args.DATASET.BASE_PATH+str(folder))[:images_in_folder]:
#             images_path.append(args.DATASET.BASE_PATH + str(folder) +'/'+ path)
    
#     random.shuffle(images_path)
#     print(" Total Images :", len(images_path))
    
#     train_ds=tf.data.Dataset.from_tensor_slices((images_path))
    
#     train = (train_ds.map(lambda x: tf.py_function(func=load_data,inp=[x], Tout=[tf.float32, tf.string]))
#             .batch(BS)
#             .prefetch(AUTO)
#            )
    
    
#     break

## Model building

In [ ]:
# Reference: https://github.com/wangz10/contrastive_loss/blob/master/model.py
class UnitNormLayer(tf.keras.layers.Layer):
    '''Normalize vectors (euclidean norm) in batch to unit hypersphere.
    '''
    def __init__(self):
        super(UnitNormLayer, self).__init__()

    def call(self, input_tensor):
        norm = tf.norm(input_tensor, axis=1)
        return input_tensor / tf.reshape(norm, [-1, 1])
def encoder_net():
    inputs = Input((args.IMAGE_SHAPE, args.IMAGE_SHAPE, 3))
    normalization_layer = UnitNormLayer()

    encoder = tf.keras.applications.ResNet50(weights=None, include_top=False)
#     encoder = tf.keras.applications.EfficientNetB3(include_top=False,weights=None)
    encoder.trainable = True

    embeddings = encoder(inputs, training=True)
    embeddings = GlobalAveragePooling2D()(embeddings)
    norm_embeddings = normalization_layer(embeddings)

    encoder_network = Model(inputs, norm_embeddings)

    return encoder_network
def projector_net():
    projector = tf.keras.models.Sequential([
        Dense(1024, activation="relu"),
        Dense(256, activation="relu"),
        UnitNormLayer()
    ])

    return projector

In [ ]:
#SGD with lr decay function
lr_decayed_fn = tf.keras.experimental.CosineDecay(initial_learning_rate= args.LR, decay_steps=args.DECAY_STEPS)
optimizer = tf.keras.optimizers.RMSprop(lr_decayed_fn) 
# tfa.optimizers.SGDW() #https://www.tensorflow.org/addons/api_docs/python/tfa/optimizers/SGDW

In [ ]:
encoder_r = encoder_net()
projector_z = projector_net()

In [ ]:
# encoder_r.build((160,160,3))
# projector_z.build((1,2048))

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        r = encoder_r(images, training=True)
        z = projector_z(r, training=True)
        loss = losses.supervised_nt_xent_loss(z, labels, temperature=0.07, base_temperature=0.07 )

    gradients = tape.gradient(loss, 
        encoder_r.trainable_variables + projector_z.trainable_variables)
    optimizer.apply_gradients(zip(gradients, 
        encoder_r.trainable_variables + projector_z.trainable_variables))

    return loss

In [ ]:
def load_data(t):
    label = str(t.numpy()).split('/')[-2]
    image_pixels = tf.image.decode_jpeg(tf.io.read_file(t))
    image_pixels = tf.image.resize(image_pixels, [160,160])  
    
    #image_pixels = tf.image.random_brightness(image_pixels,max_delta=0)
    #image_pixels = tf.image.random_contrast(image_pixels,lower=0.2, upper=0.8)
    #image_pixels = tf.image.random_saturation(image_pixels, lower=0.2, upper=0.6)
    
    return image_pixels, label #return t.numpy().decode('utf-8').upper()
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
def plot_embeddings(emb,labels):
    tl=TSNE()
    embedding=tl.fit_transform(emb)
    fig = plt.figure(figsize = (10, 10))
    sns.scatterplot(embedding[:,0], embedding[:,1], hue=labels)
    plt.show()

    return fig

In [ ]:
wandb.init(project="Contrastive_FacesEmore",id='Resnet50_0.001')
train_loss_results = []


for epoch in range(args.EPOCHS):
    start = time.time()
    epoch_loss_avg = tf.keras.metrics.Mean()
    
    for cur_class in range(0,args.DATASET.TOTAL_CLASSES, args.NUMBER_OF_PEOPLE):
        images_path = []
        for folder in range(cur_class, cur_class + args.NUMBER_OF_PEOPLE):
            images_in_folder = len(os.listdir(args.DATASET.BASE_PATH+str(folder)))
            images_in_folder = images_in_folder if images_in_folder < args.NUMBER_IMAGES_PER_PEOPLE else args.NUMBER_IMAGES_PER_PEOPLE
            for path in os.listdir(args.DATASET.BASE_PATH+str(folder))[:images_in_folder]:
                images_path.append(args.DATASET.BASE_PATH + str(folder) +'/'+ path)

        random.shuffle(images_path)

        train_ds=tf.data.Dataset.from_tensor_slices((images_path))

        train = (train_ds.map(lambda x: tf.py_function(func=load_data,inp=[x], Tout=[tf.float32, tf.string]))
                .batch(args.BATCH_SIZE)
                .prefetch(AUTO)
               )
        
        logging.info(" Total Images  :"+  str(len(images_path)) + '\n')
        logging.info(" Current CHUNK :"+  str(cur_class) +  "  " + str(cur_class + args.NUMBER_OF_PEOPLE) + '\n')
        
        print(" Total Images  :", len(images_path))
        print(" Current CHUNK :", str(cur_class)+ " " + str(cur_class + args.NUMBER_OF_PEOPLE))
        print(" Steps in Epoch:", int(len(images_path)/args.BATCH_SIZE))
        
        for step , (images, labels) in enumerate(train):
            loss = train_step(images, labels)
            epoch_loss_avg.update_state(loss) 
            
            if step % args.LOG_EVERY == 0:
                logging.info(" EPOCH :" + str(epoch) + "  Step : " + str(step) + "  Loss : " + str(loss) + '\n')
                print( " EPOCH :" , epoch, "Step : ", step, "Loss : ", loss)
                wandb.log({"supervised_contrastive_loss": epoch_loss_avg.result()})
                
        #if epoch % args.PLOT == 0:
        #    res = encoder_r(images)
        #    fig = plot_embeddings(res,labels.numpy())
        #    wandb.log(({"Embedding Plot " + str(epoch) + '/' + str(args.EPOCHS) : wandb.Image(fig)}))

        
        if epoch % args.CHECKPOINT_EVERY == 0:
            encoder_r.save_weights(args.SAVE_DIR+'/encoder/weights')
            projector_z.save_weights(args.SAVE_DIR+'/projector/weights')
        
        train_loss_results.append(epoch_loss_avg.result())
        

        break
#     break
    print( " Time Taken :" time.time() - start )

In [ ]:
with plt.xkcd():
    plt.plot(train_loss_results)
    plt.title("Supervised Contrastive Loss")
    plt.show()

In [ ]:
train

In [ ]:
np.unique(labels.numpy())

In [ ]:
labels.numpy()

In [ ]:
labels.numpy()[19]

In [ ]:
labels.numpy()[20]

In [ ]:
a = res[19].numpy()
b = res[21].numpy()

In [ ]:
from numpy import dot
from numpy.linalg import norm

cos_sim = dot(a, b)/(norm(a)*norm(b))

In [ ]:
cos_sim

In [ ]:


res=encoder_r(images)
fig = plot_embeddings(res,labels.numpy())

In [ ]:
rr = projector_z(res)

In [ ]:
rr[0]

In [ ]:
images.shape, labels.shape

In [ ]:
labels

In [ ]:
res = encoder_r(images)

In [ ]:
res.shape